In [11]:
from pathlib import Path
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from tqdm.auto import tqdm, trange



import string
import keras
from keras.preprocessing.sequence import pad_sequences  # new!
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout,SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.layers import Embedding  # новая!
from keras.callbacks import ModelCheckpoint  # новая!
import os  # новая!
from sklearn.metrics import roc_auc_score, roc_curve  # новая!
import matplotlib.pyplot as plt  # новая!
import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec

from sklearn.manifold import TSNE

import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure


In [ ]:
from ann_visualizer.visualize import ann_viz

In [ ]:
import numpy as np

samples = ['What is the answer to everything?']
token_index = {}

for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
max_length = 6
results = np.zeros(shape=(len(samples),
                          max_length,
                          max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

results

In [ ]:
# имя каталога для сохранения результатов:
output_dir = 'model_output/dense'
# обучение:
epochs = 4
batch_size = 128
# параметры векторного пространства слов:
n_dim = 64
n_unique_words = 5000
n_words_to_skip = 50
max_review_length = 100
pad_type = trunc_type = 'pre'
# архитектура полносвязанной сети:
n_dense = 64
dropout = 0.5

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim,input_length=max_review_length))
model.add(Flatten())
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(3, activation='softmax'))
model.summary()



In [ ]:
epochs = 4
batch_size = 128
# векторное пространство слов:
n_dim = 64
n_unique_words = 5000
max_review_length = 400
pad_type = trunc_type = 'pre'
drop_embed = 0.2 # новое!
# архитектура сверточного слоя:
n_conv = 256 # фильтры, они же ядра
k_conv = 3 # длина ядра
# архитектура полносвязанного слоя:
n_dense = 256
dropout = 0.2

In [ ]:
model = Sequential()
# Векторное пространство слов:
model.add(Embedding(n_unique_words, n_dim,
 input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
# сверточный слой:
model.add(Conv1D(n_conv, k_conv, activation='relu'))
# model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(GlobalMaxPooling1D())
# полносвязанный слой:
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
# выходной слой:
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 4
batch_size = 128
# векторное пространство слов:
n_dim = 64
n_unique_words = 5000
max_review_length = 400
pad_type = trunc_type = 'pre'
drop_embed = 0.2 # новое!
# архитектура сверточного слоя:
n_conv = 256 # фильтры, они же ядра
k_conv = 3 # длина ядра
# архитектура полносвязанного слоя:
n_dense = 256
dropout = 0.2

In [ ]:
model = Sequential()
# Векторное пространство слов:
model.add(Embedding(n_unique_words, n_dim,
 input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
# сверточный слой:
model.add(Conv1D(n_conv, k_conv, activation='relu'))
# model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(GlobalMaxPooling1D())
# полносвязанный слой:
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
# выходной слой:
model.add(Dense(3, activation='softmax'))

In [ ]:
# имя каталога для сохранения результатов:
output_dir = 'model_output/LSTM'
# обучение:
epochs = 4
batch_size = 128
# параметры векторного пространства слов:
n_dim = 64
n_unique_words = 10000
max_review_length = 100
pad_type = trunc_type = 'pre'
drop_embed = 0.2
# архитектура сети LSTM:
n_lstm = 256
drop_lstm = 0.2

In [ ]:
from keras.layers import LSTM, GRU
model = Sequential()
model.add(Embedding(n_unique_words, n_dim,
 input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
model.add(GRU(n_lstm, dropout=drop_lstm))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
ann_viz(model,  view=True)

In [3]:
model = Sequential()
# Векторное пространство слов:
model.add(Embedding(n_unique_words, n_dim,
 input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
# сверточный слой:
model.add(Conv1D(n_conv, k_conv, activation='relu'))
# model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(GlobalMaxPooling1D())
# полносвязанный слой:
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
# выходной слой:
model.add(Dense(3, activation='softmax'))

In [12]:
# имя каталога для сохранения результатов:
output_dir = 'model_output/LSTM'
# обучение:
epochs = 4
batch_size = 128
# параметры векторного пространства слов:
n_dim = 64
n_unique_words = 10000
max_review_length = 100
pad_type = trunc_type = 'pre'
drop_embed = 0.2
# архитектура сети LSTM:
n_lstm = 256
drop_lstm = 0.2

In [17]:
from keras.layers import LSTM, GRU
model = Sequential()
model.add(Embedding(n_unique_words, n_dim,
 input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
model.add(GRU(n_lstm, dropout=drop_lstm))
model.add(Dense(3, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 64)           640000    
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 100, 64)          0         
 lDropout1D)                                                     
                                                                 
 gru (GRU)                   (None, 256)               247296    
                                                                 
 dense_6 (Dense)             (None, 3)                 771       
                                                                 
Total params: 888,067
Trainable params: 888,067
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [14]:
ann_viz(model,  view=True)

ValueError: ANN Visualizer: Layer not supported for visualizing

In [48]:
path_to_books = Path('D:/Dev/GraduateProject/raw_books/')
stpwrds = stopwords.words('russian') + list(string.punctuation)


In [49]:
books = []
for x in path_to_books.rglob('*.txt'):
    with open(x, 'r', encoding='utf-8') as r:
        data = str(r.read())

    books.append(data.replace('\xa0', '').replace('\t', ''))


In [50]:
tk_books = []
for book in books:
    tk_books.append(sent_tokenize(book))

In [63]:
len(tk_books[2])

25762

In [53]:
data = {'author': [], 'name': [], 'text': []}
for book in tk_books:
    data['author'].append(book[0])
    data['name'].append(book[1])
    data['text'].append(book[2:])
df = pd.DataFrame(data)

In [57]:
df.to_csv('dataset.csv')

In [56]:
df['tag'] = [0, 1, 2]
# 0 - фантастика
# 1 - детектив
# 2 - фэнтези

In [78]:
df['text']

0    [Глава 1., Дом стоял на склоне холма на окpаин...
1    [1\nБыла половина октября, около одиннадцати у...
2    [ПРОЛОГ\n\n1., О ХОББИТАХ., Рассказ у нас пойд...
Name: text, dtype: object

In [76]:
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])


def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])


import re


def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

In [85]:

for i, record in df.iterrows():
    print(record)

author                                        Адамс Дуглас.
name      Путеводитель по Галактике для путешествующих а...
text      [Глава 1., Дом стоял на склоне холма на окpаин...
tag                                                       0
Name: 0, dtype: object
author                                     Чэндлеp Раймонд.
name                                          Глубокий сон.
text      [1\nБыла половина октября, около одиннадцати у...
tag                                                       1
Name: 1, dtype: object
author                                      Дж.Р.Р.Толкиен.
name                                       ВЛАСТЕЛИН КОЛЕЦ.
text      [ПРОЛОГ\n\n1., О ХОББИТАХ., Рассказ у нас пойд...
tag                                                       2
Name: 2, dtype: object


In [92]:
texts = {'sentences': [], 'tag': []}
for i, record in tqdm(df.iterrows()):
    for sent in record['text']:
        texts['sentences'].append(' '.join(
            [remove_multiple_spaces(remove_numbers(remove_punctuation(w.lower()))) for w in word_tokenize(sent) if
             w not in stpwrds]))
        texts['tag'].append(record['tag'])

0it [00:00, ?it/s]

In [93]:
df_res = pd.DataFrame(texts)

In [94]:
df_res

,sentences,tag
0,глава,0
1,дом стоял склоне холма окpаине гоpодка отдельн...,0
2,окна выходили шиpокую pавнину западной англии,0
3,с стоpоны посмотpеть дом тpидцать лет это квад...,0
4,единственный кого дом чем то устpаивал аpтуp д...,0
...,...,...
36079,а сэм взял путь приречье подъехал круче закат ...,2
36080,прощальные бледно золотистые лучи озарили торб...,2
36081,роза встретила подвинула кресло камину усадила...,2
36082,он глубоко вздохнул,2


In [96]:
df_res.to_csv('dataset_sent.csv')